In [1]:
import warnings
warnings.filterwarnings('ignore')
#Librerias para filtrar warnings
import pandas as pd
import numpy as np
#Libreria de analisis numerico
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (10, 6)
#librerias de graficos
from scipy import stats
#Librerias estadisticas
import statsmodels.api as sm
import statsmodels.formula.api as smf
#librerias de estadsitica aplicada

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures
#Librerias espeficias para regresion logistica, su validadion, presicion y evaluar su eficacia.
#Pueden faltar librerias para temas mas especificos y al a vez librerias util que permitan funciones adicionales
from functions import printUniques , replaceAll , concise_summary,logitCalculate

In [2]:
df = pd.read_csv('income-db.csv')
df_filter = df

In [3]:
df_filter = replaceAll(df_filter,'?',np.nan)
df_filter = replaceAll(df_filter,['Exec-managerial','Prof-specialty','Adm-clerical','Sales','Tech-support'],'white_collar')
df_filter = replaceAll(df_filter,['Craft-repair','Machine-op-inspct','Transport-moving','Handlers-cleaners','Farming-fishing','Protective-serv','Priv-house-serv'],'blue_collar')
df_filter = replaceAll(df_filter,['Other-service','Armed-Forces'],'others')
df_filter = replaceAll(df_filter,['Federal-gov'],'federal_gov')
df_filter = replaceAll(df_filter,['State-gov','Local-gov'],'state_level_gov' )
df_filter = replaceAll(df_filter,['Self-emp-inc','Self-emp-not-inc'],'self_employ' )
df_filter = replaceAll(df_filter,['Never-worked','Without-pay'],'unemployed' )
df_filter = replaceAll(df_filter,['Preschool'],'preschool')
df_filter = replaceAll(df_filter,['1st-4th','5th-6th'],'elementary_school')
df_filter = replaceAll(df_filter,['7th-8th','9th','10th','11th','12th','HS-grad'],'high_school')
df_filter = replaceAll(df_filter,['Assoc-voc','Assoc-acdm','Some-college'],'college')
df_filter = replaceAll(df_filter,['Bachelors','Masters','Prof-school','Doctorate'],'university')
df_filter = replaceAll(df_filter,['Married-civ-spouse','Married-spouse-absent','Married-AF- spouse'],'married')
df_filter = replaceAll(df_filter,['Divorced'],'divorced')
df_filter = replaceAll(df_filter,['Separated'],'separated')
df_filter = replaceAll(df_filter,['Widowed'],'widowed')
df_filter = replaceAll(df_filter,['United-States','Mexico','Puerto-Rico','Canada','El-Salvador','Cuba','Jamaica','Dominican-Republic','Guatemala','Columbia','Haiti','Nicaragua','Peru','Ecuador','Trinadad&Tobago','Outlying-US(Guam-USVI-etc)','Honduras'],'America')
df_filter = replaceAll(df_filter,['England','Portugal','Italy','Germany','Poland','Greece','Ireland','France','Scotland','Yugoslavia','Hungary','Holand-Netherlands'],'Europe')
df_filter = replaceAll(df_filter,['Philippines','India','China','Japan','South','Vietnam','Iran','Taiwan','Thailand','Hong','Cambodia','Laos'],'Asia')
df_filter = replaceAll(df_filter,['<=50K','>50K'], [0,1])
df_filter = replaceAll(df_filter,'-','_')


In [4]:
df_filter['collars'] = df_filter['occupation']
df_filter = df_filter.drop('occupation', axis=1)
df_filter['workclass_recod'] = df_filter['workclass']
df_filter = df_filter.drop('workclass', axis=1)
df_filter['education_recod'] = df_filter['education']
df_filter = df_filter.drop('education', axis=1)
df_filter['civstatus'] = df_filter['marital-status']
df_filter = df_filter.drop('marital-status', axis=1)

In [5]:
df_dummy = pd.get_dummies(df_filter)

In [6]:
df_dummy = df_dummy.rename(index=str, columns={'capital-gain':'capital_gain',
                                               'capital-loss':'capital_loss',
                                               'educational-num':'educational_num',
                                               'hours-per-week':'hours_per_week',
                                               'relationship_Not-in-family':'relationship_Not_in_family',
                                              'relationship_Other-relative':'relationship_Other_relative',
                                              'relationship_Own-child':'relationship_Own_child',
                                              'race_Amer-Indian-Eskimo':'race_Amer_Indian_Eskimo',
                                              'race_Asian-Pac-Islander':'race_Asian_Pac_Islander',
                                              'native-country_America':'native_country_America',
                                              'native-country_Asia':'native_country_Asia',
                                              'native-country_Europe':'native_country_Europe',
                                              'civstatus_Married-AF-spouse':'civstatus_Married_AF_spouse',
                                              'civstatus_Never-married':'civstatus_Never_married'})

In [7]:
df_dummy.columns.values

array(['age', 'fnlwgt', 'educational_num', 'capital_gain', 'capital_loss',
       'hours_per_week', 'income', 'relationship_Husband',
       'relationship_Not_in_family', 'relationship_Other_relative',
       'relationship_Own_child', 'relationship_Unmarried',
       'relationship_Wife', 'race_Amer_Indian_Eskimo',
       'race_Asian_Pac_Islander', 'race_Black', 'race_Other',
       'race_White', 'gender_Female', 'gender_Male',
       'native_country_America', 'native_country_Asia',
       'native_country_Europe', 'collars_blue_collar', 'collars_others',
       'collars_white_collar', 'workclass_recod_Private',
       'workclass_recod_federal_gov', 'workclass_recod_self_employ',
       'workclass_recod_state_level_gov', 'workclass_recod_unemployed',
       'education_recod_college', 'education_recod_elementary_school',
       'education_recod_high_school', 'education_recod_preschool',
       'education_recod_university', 'civstatus_Married_AF_spouse',
       'civstatus_Never_married', '

# La modelación mediante regresión de ambos problemas. 

In [8]:
df_dummy.dropna(how='any' , inplace=True)

In [9]:
logit_reg = smf.logit('income ~ age + fnlwgt + educational_num + capital_gain + capital_loss + hours_per_week + relationship_Husband + relationship_Other_relative + relationship_Own_child  + relationship_Unmarried + relationship_Wife + race_Amer_Indian_Eskimo + race_Asian_Pac_Islander + race_Black + race_White + gender_Male + native_country_America + native_country_Europe + collars_blue_collar + collars_white_collar + workclass_recod_Private + workclass_recod_federal_gov + workclass_recod_self_employ + workclass_recod_state_level_gov + education_recod_college + education_recod_elementary_school + education_recod_high_school + education_recod_university + civstatus_Married_AF_spouse + civstatus_Never_married + civstatus_divorced + civstatus_married + civstatus_separated', df_dummy).fit()
logit_reg.summary()

Optimization terminated successfully.
         Current function value: 0.323623
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 income   No. Observations:                48842
Model:                          Logit   Df Residuals:                    48808
Method:                           MLE   Df Model:                           33
Date:                Wed, 10 Jul 2019   Pseudo R-squ.:                  0.4119
Time:                        22:26:31   Log-Likelihood:                -15806.
converged:                       True   LL-Null:                       -26875.
                                        LLR p-value:                     0.000
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           -13.6900      3.933     -3.481      0.000     -21.398      -5.982
age                                   0.0252      0.001     19.321      0.000       0.023       0.028
fnlwgt                             7.276e-07   1.37e-07      5.325      0.000     4.6e-07    9.95e-07
educational_num                       0.2749      0.015     17.787      0.000       0.245       0.305
capital_gain                          0.0003   8.35e-06     37.973      0.000       0.000       0.000
capital_loss                          0.0007   2.98e-05     21.968      0.000       0.001       0.001
hours_per_week                        0.0294      0.001     23.337      0.000       0.027       0.032
relationship_Husband                  0.9638      0.128      7.512      0.000       0.712       1.215
relationship_Other_relative          -0.5294      0.172     -3.083      0.002      -0.866      -0.193
relationship_Own_child               -0.9791      0.114     -8.586      0.000      -1.203      -0.756
relationship_Unmarried               -0.2430      0.082     -2.961      0.003      -0.404      -0.082
relationship_Wife                     2.0327      0.142     14.324      0.000       1.755       2.311
race_Amer_Indian_Eskimo              -0.2227      0.261     -0.852      0.394      -0.735       0.290
race_Asian_Pac_Islander               0.3632      0.216      1.682      0.093      -0.060       0.786
race_Black                            0.0191      0.203      0.094      0.925      -0.378       0.416
race_White                            0.2804      0.195      1.440      0.150      -0.101       0.662
gender_Male                           0.7708      0.063     12.249      0.000       0.647       0.894
native_country_America                0.2740      0.094      2.923      0.003       0.090       0.458
native_country_Europe                 0.4754      0.139      3.426      0.001       0.203       0.747
collars_blue_collar                   0.7102      0.084      8.429      0.000       0.545       0.875
collars_white_collar                  1.3075      0.083     15.739      0.000       1.145       1.470
workclass_recod_Private              -0.0995      0.114     -0.871      0.384      -0.324       0.125
workclass_recod_federal_gov           0.3549      0.134      2.644      0.008       0.092       0.618
workclass_recod_self_employ          -0.4370      0.119     -3.677      0.000      -0.670      -0.204
workclass_recod_state_level_gov      -0.1740      0.120     -1.453      0.146      -0.409       0.061
education_recod_college               3.9870      3.925      1.016      0.310      -3.706      11.680
education_recod_elementary_school     4.1601      3.927      1.059      0.289      -3.537      11.857
education_recod_high_school           3.9075      3.924      0.996      0.319      -3.784      11.599
education_recod_university            4.0626      3.927      1.035      0.301  

In [10]:
concise_summary(logit_reg)


Goodnes of Fit statistics
           Statistics       Value
2              Date:  31979.8245
3  No. Observations:     -15806.
4          Df Model:     -26875.
5      Df Residuals:      0.0000
6         Converged:      1.0000
7    No. Iterations:            

Point Estimates

                                           Coef.      Std.Err.
Intercept                         -1.369003e+01  3.932549e+00
age                                2.523265e-02  1.305961e-03
fnlwgt                             7.275955e-07  1.366411e-07
educational_num                    2.748534e-01  1.545262e-02
capital_gain                       3.171931e-04  8.353078e-06
capital_loss                       6.546416e-04  2.979988e-05
hours_per_week                     2.942534e-02  1.260888e-03
relationship_Husband               9.637723e-01  1.282974e-01
relationship_Other_relative       -5.293640e-01  1.717292e-01
relationship_Own_child            -9.790721e-01  1.140261e-01
relationship_Unmarried            -2.430

# La interpretación de los principales regresores en cada problema. 

In [11]:
params =  ['age','fnlwgt','educational_num','capital_gain','capital_loss','hours_per_week','relationship_Husband','relationship_Other_relative','relationship_Own_child','relationship_Unmarried','relationship_Wife','gender_Male','native_country_America','native_country_Europe','collars_blue_collar','collars_white_collar','workclass_recod_federal_gov','workclass_recod_self_employ','civstatus_Married_AF_spouse','civstatus_Never_married','civstatus_married']
logitCalculate(params, logit_reg)


0.9977076077163946

al mantener los regresores al 95% de confianza, mantenemos un resultado similar en confianza del modelo para este modelo, tendremos un resultado muy "FIT" para el modelo, en lo pronto tendra que ser modificado

# La definición de las estrategias de Modelación predictiva

Como estrategia para el aprendizaje de la maquina, se establecera un fit segun los resultados donde nos acompañara la librerias de sklearn aunque ya existen algunos regresores que nos apoyan para los resultados que buscamos.
Sera recomendable utilizar practicamente todos los datos en la variable params